In [ ]:
#%reset
#import sys
#sys.path.append("/home/chemical/phd/chz198152/apps/msm_we")
from msm_we import msm_we
import mdtraj as md
import tqdm
import numpy as np
import pyemma
import matplotlib.pyplot as plt
import ray

In [ ]:
file_paths = ["./west.h5"]
ref_structure = "./random_centered_seg.pdb"

In [ ]:
def processCoordinates(self, coords):
    if self.dimReduceMethod == "none":
        nC = np.shape(coords)
        nC = nC[0]
        data = coords.reshape(nC, 3*self.nAtoms)
        return data
    elif self.dimReduceMethod == "pca":
        xt = md.Trajectory(xyz=coords, topology=None)
        indCA = self.reference_structure.topology.select('index 0 to 38')
        pair1, pair2 = np.meshgrid(indCA, indCA, indexing="xy")
        indUT = np.where(np.triu(pair1, k=1) > 0)
        pairs = np.transpose(np.array([pair1[indUT], pair2[indUT]])).astype(int)
        dist = md.compute_distances(xt, pairs, periodic=True, opt=True)
        
        return dist
msm_we.modelWE.processCoordinates = processCoordinates

In [ ]:
ray.init(num_cpus=4)#, ignore_reinit_error=True)

In [ ]:
model = msm_we.modelWE()
model.initialize(
    file_paths,
    ref_structure,
    modelName='bispidine',
    basis_pcoord_bounds = [[50, 130], [50, 120]],
    target_pcoord_bounds = [[-130, -50], [50,120]],
    dim_reduce_method = 'pca',
    tau = 1,
    pcoord_ndim=2,
)

In [ ]:
model.get_iterations()

In [ ]:
model.get_coordSet(last_iter=model.maxIter, streaming=True)

In [ ]:
model.dimReduce()

In [ ]:
model.cluster_coordinates(n_clusters=1, first_cluster_iter=1, streaming=True, stratified=True, use_ray=True)

In [ ]:
model.get_fluxMatrix(0, first_iter=0, last_iter=model.maxIter, use_ray=False)

In [ ]:
model.organize_fluxMatrix(use_ray=False)

In [ ]:
model.get_Tmatrix()

In [ ]:
model.get_steady_state()
model.get_steady_state_target_flux()

In [ ]:
print(f"Steady-state target flux is {model.JtargetSS:.2e}")

In [ ]:
model.plot_flux(suppress_validation=True)

In [ ]:
model.get_committor()
model.plot_committor()

In [ ]:
model.plot_flux_committor(suppress_validation=True)
plt.gca().set_xscale('linear')

## Modify h5 files
Removes solvent coordinates

In [ ]:
import h5py
import numpy as np

In [ ]:
f1 = h5py.File('bispidine-fluxmatrix-_s1_e200_lag0_clust2592.h5', 'r+')

In [ ]:
fluxmatrix = np.array(f1['fluxMatrix'])

In [ ]:
fluxmatrix_shape = np.shape(fluxmatrix)

In [ ]:
where = np.where(fluxmatrix !=0)

In [ ]:
plt.plot(x)

In [ ]:
fluxes_out = np.sum(fluxmatrix, 1)

In [ ]:
for state_idx in range(fluxmatrix_shape[0]):
    if fluxes_out[state_idx] > 0:
        fluxmatrix[state_idx, :] = (fluxmatrix[state_idx, :] / fluxes_out[state_idx])
    if fluxes_out[state_idx] == 0.0:
        fluxmatrix[state_idx, state_idx] = 1.0

## Direct model

In [ ]:
%reset
import sys
sys.path.append("/home/chemical/phd/chz198152/apps/msm_we")
from msm_we import msm_we
import mdtraj as md
import tqdm
import numpy as np
import pyemma
import matplotlib.pyplot as plt
import ray

file_paths = ["./west.h5"]
ref_structure = "./random_centered_seg.pdb"

In [ ]:
def processCoordinates(self, coords):
    if self.dimReduceMethod == "none":
        nC = np.shape(coords)
        nC = nC[0]
        data = coords.reshape(nC, 3*self.nAtoms)
        return data
    elif self.dimReduceMethod == "pca":
        xt = md.Trajectory(xyz=coords, topology=None)
        indCA = self.reference_structure.topology.select('index 0 to 38')
        pair1, pair2 = np.meshgrid(indCA, indCA, indexing="xy")
        indUT = np.where(np.triu(pair1, k=1) > 0)
        pairs = np.transpose(np.array([pair1[indUT], pair2[indUT]])).astype(int)
        dist = md.compute_distances(xt, pairs, periodic=True, opt=True)
        
        return dist
msm_we.modelWE.processCoordinates = processCoordinates

In [ ]:
model = msm_we.modelWE()

model.build_analyze_model(
    ray_kwargs = {'num_cpus': 4, 'include_dashboard': False},
    file_paths = file_paths,
    ref_struct = ref_structure,
    modelName='bispidine',
    basis_pcoord_bounds = [[50, 130],[50, 130]],
    target_pcoord_bounds = [[-130, -50],[50, 130]],
    dimreduce_method = 'pca',
    tau = 1,
    n_clusters = 2,
    stratified=True,
    streaming=True,
    use_ray=True,
    show_live_display=True,
    fluxmatrix_iters=[1,-1]
)

In [ ]:
model.pcoord_ndim